<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Hugging_Face_Sara_Test_RICHTIGE%20ANTWORT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy
!pip uninstall -y tensorflow


In [2]:
# Imports
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import sklearn
import torch
import time

In [3]:
from google.colab import userdata
HuggingFaceAPIKey = "HF2"

In [4]:
# PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [ ]:
# Text in Chunks teilen
def split_text(text, chunk_size=300, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


In [7]:
# Embedding Modell laden
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Optimale Batch-Größe finden (Experimentieren!)
optimal_batch_size = 5000  # Starte mit einem hohen Wert und reduziere, wenn du Speicherprobleme hast

def create_faiss_index(chunks):
    start_time = time.time()  # Zeitmessung

    # Embeddings erstellen (mit optimierter Batch-Verarbeitung und Tensor-Konvertierung)
    chunk_embeddings_tensor = embedding_model.encode(chunks, batch_size=optimal_batch_size, convert_to_tensor=True)
    chunk_embeddings = chunk_embeddings_tensor.cpu().numpy()

    # Normalisieren der Embeddings (wichtig für die meisten Distanzmetriken)
    chunk_embeddings = sklearn.preprocessing.normalize(chunk_embeddings)

    dimension = chunk_embeddings.shape[1]

    # FAISS Index erstellen (IndexIVFFlat mit Training)
    nlist = int(np.sqrt(len(chunk_embeddings)))  # Anzahl der Partitionen (Faustregel)
    quantizer = faiss.IndexFlatL2(dimension)  # Quantisierer für die Clusterzentren
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

    # Training des Index (NOTWENDIG für IndexIVFFlat!)
    index.train(chunk_embeddings)

    # Hinzufügen der Embeddings zum Index (NACH dem Training)
    index.add(chunk_embeddings)

    end_time = time.time()
    print(f"FAISS Vektordatenbank erstellt! (Zeit: {end_time - start_time:.2f} Sekunden)")
    return index

index = create_faiss_index(chunks)



FAISS Vektordatenbank erstellt! (Zeit: 160.57 Sekunden)


In [15]:
def search(index, query, k=5):
    query_embedding_tensor = embedding_model.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding_tensor.cpu().numpy()

    query_embedding = sklearn.preprocessing.normalize(query_embedding)
    D, I = index.search(query_embedding, k)  # D: Distanzen, I: Indizes
    return D, I

def get_search_results(index, query, chunks, k=5):
    D, I = search(index, query, k)
    results = []
    for i, distance in zip(I[0], D[0]):
        results.append({"chunk": chunks[i], "distance": distance})
    return results

In [16]:
# Ähnlichkeitssuche in FAISS
def get_relevant_chunks(query, top_k=3):
    # Get the embedding using the pre-loaded embedding_model
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

!huggingface-cli login

os.environ["HUGGINGFACE_API_KEY"] = "HF2"
model_name = "mistralai/Mistral-7B-v0.1"  # Alternativ: "meta-llama/Llama-2-7b-chat-hf"

# Tokenizer & Modell mit explizitem Token laden
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")


def generate_answer(query):
    relevant_chunks = get_relevant_chunks(query, top_k=10)
    context = "\n".join(relevant_chunks)
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n\n{context}\n\nFrage: {query}\nAntwort:"

    response = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.5)
    return response[0]["generated_text"]


In [18]:
# Test
frage = "Wie hoch ist der gesetzliche Mindestlohn?"
antwort = generate_answer(frage)
print("Antwort:", antwort)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Antwort: Beantworte die Frage basierend auf diesem Kontext:

ch im Krankheitsfall zu entrichten (BAG – 5 AZR 457/00).
3.9  Vergütung

86
3.9.3	
Gesetzlicher Mindestlohn
Rechtsgrundlage ist das Mindestlohngesetz (MiLoG). Darin ist ein gesetzlicher Anspruch 
des Arbeitnehmers auf einen Mindestlohn geregelt, die eigenständig neben den arbeits- 
oder tarifvertra
tiert 
sich nicht an den Vorgaben des gesetzlichen Mindestlohns (vgl. hierzu auch BT-Drs. 
18/2010 (neu), S. 24)). ◄
Branchenspezifische Mindestlohnbestimmungen sind vorrangig, z. B. bei Arbeitsüber-
lassung oder Zeitarbeit (§ 3a AÜG) oder bei Arbeitnehmer-Entsendung (§§ 4, 7a AEntG).
3  Inhalt des 
hend muss im Wahlausschreiben die Mindestzahl von Sitzen angegeben werden, 
die dem Minderheitsgeschlecht zustehen.
Dem Betriebsrat kommt als Institution keine eigene Rechtsfähigkeit zu (BGH – III ZR 
266/11).
5.5.3	
Arbeitsgrundsätze
Nach § 2 BetrVG gilt der Grundsatz der vertrauensvollen Zusammen
 Betriebsrat vertreten sein, 
wenn d

In [19]:
# Test
frage = "Wie hoch ist die Kündigungsfrist in der Probezeit?"
antwort = generate_answer(frage)
print("Antwort:", antwort)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Antwort: Beantworte die Frage basierend auf diesem Kontext:

ernde Wirkungen hätte. Ist die Kündigung wirksam? (AG Düsseldorf – 7 Ca 
2591/11). ◄
Bei Auszubildenden sind besondere Schutzbedürfnisse zu beachten, sodass eine frist-
lose Kündigung aus wichtigem Grund nur das allerletzte Mittel darstellen darf. Selbst im 
Falle schlechter Arbeitsleistungen darf e
schläge zu den Kontrollfragen

192
6.2.4	
Kapitel 4
	1.	 Zu einer wirksamen Kündigungserklärung ist eine bestimmte und unmissverständliche 
Erklärung des Kündigenden erforderlich. Diese muss auch Angaben zum Beendigungs­
datum enthalten. Ggf. reicht ein Hinweis auf spezielle Kündigungsfristen aus V
cht funktio-
nieren könne, weshalb er mit der verkürzten Kündigungsfrist kündigt. AN meint, die 
Kündigung sei unwirksam, weil sich AG nicht genügend Zeit für eine Einschätzung 
ihrer Arbeit gelassen habe. Ist die Kündigung wirksam (vgl. hierzu LAG München -, 9 
Sa 406/05)? ◄
Abweichendes kann sich 
Frist beendet. Eine solche Kündigung